# Zero Curve Construction and Analysis

## Overview
This notebook demonstrates the construction and manipulation of zero curves, which are fundamental tools in fixed income mathematics and financial engineering. The notebook covers:

1. Implementation of exponential interpolation for yield curves
2. Working with the ZeroCurve class to:
   - Add zero rates to the curve
   - Calculate discount factors
   - Perform interpolation for missing maturities
   - Transform curve data for analysis

## Key Concepts
- **Zero Rates**: Interest rates for zero-coupon bonds of different maturities
- **Discount Factors**: Present values of 1 unit of currency paid at future dates
- **Exponential Interpolation**: A method for estimating rates between known points on the curve
- **Amount at Maturity (AtMat)**: Future value based on zero rates

## Dependencies
The notebook uses standard Python libraries for mathematical operations and data manipulation:
- math: For basic mathematical operations
- numpy: For numerical computations
- pandas: For data manipulation and analysis
- Custom yield_curve module: Contains ZeroCurve class implementation

In [1]:
import math
import numpy as np
import importlib
import pandas as pd

# Exponential Interpolation Demonstration

The following section demonstrates exponential interpolation, which is a key technique in yield curve construction. This method is particularly suitable for interest rates because:

1. It preserves the positivity of rates
2. It reflects the compound growth nature of interest rates
3. It provides smooth and continuous interpolation between known points

Below, we'll first implement the interpolation manually to understand the process, then use our implemented function from the yield_curve module.

In [2]:
# Exponential interpolation demonstration
# We'll interpolate between two known points (x1,y1) and (x2,y2)
# to find a value y at point x

x1 = 1
x2 = 2
y1 = 1.1    # Amount at maturity after 1 year
y2 = 1.21   # Amount at maturity after 2 years
x = 1.5     # Target interpolation point

# Calculate continuous compounding interest rate between y1 and y2
# Using the formula: r = ln(y2/y1)/(x2-x1)
rate = math.log(y2 / y1) / (x2 - x1)
print("Continuous compounding rate is: " + str(rate)) 

# Calculate the interpolated y value at point x
# Using the formula: y = y1 * exp(r * (x - x1))
y = y1 * math.exp(rate * (x - x1))

print("The Amount at Maturity at " + str(x) + " is " + str(y))


Continuous compounding rate is: 0.09531017980432474
The Amount at Maturity at 1.5 is 1.1536897329871667


## Using the Implemented Interpolation Function

Now we'll use our implemented `exp_interp` function from the yield_curve module to perform the same interpolation. This function:
1. Takes arrays of x and y values, plus the target x value
2. Finds the appropriate interval for interpolation
3. Applies exponential interpolation using the same methodology as above
4. Handles edge cases and error conditions

In [3]:
# Demonstrate exp_interp function usage

from yield_curve import exp_interp

xs = [0, 1, 2]           # Maturities
ys = [1, 1.1, 1.21]      # Amounts at maturity
x = 1.5                  # Target maturity

interpolated_y = exp_interp(xs, ys, x)
print("Interpolated y:", interpolated_y)

Interpolated y: 1.1536897329871667


# Working with the ZeroCurve Class

The following section demonstrates the main functionality of the ZeroCurve class, which provides a complete toolkit for working with zero curves. We'll explore:

1. Creating and populating a zero curve
2. Calculating various measures (zero rates, discount factors, amounts at maturity)
3. Interpolating values for non-standard maturities
4. Presenting the curve data in different formats

The ZeroCurve class manages internal consistency between different representations of the same curve (zero rates, discount factors, and amounts at maturity) and provides interpolation capabilities for missing points.

In [4]:
import yield_curve as zCurve
importlib.reload(zCurve)

# Create an instance of the ZeroCurve class
# This initializes an empty curve ready to accept rate information
zc = zCurve.ZeroCurve()

# Add zero rates to the curve
# These rates represent annual interest rates for zero-coupon bonds
zc.add_zero_rate(1, 0.01)  # 1% for 1 year
zc.add_zero_rate(2, 0.02)  # 2% for 2 years
zc.add_zero_rate(3, 0.03)  # 3% for 3 years
zc.add_zero_rate(4, 0.04)  # 4% for 4 years

# Demonstrate retrieving zero rates
print("1-year zero rate:", zc.get_zero_rate(1))

# Calculate and display discount factors
# Discount factors represent the present value of 1 unit of currency
print("1-year discount factor:", zc.get_discount_factor(1))
print("2-year discount factor:", zc.get_discount_factor(2))
print("3-year discount factor:", zc.get_discount_factor(3))
print("4-year discount factor:", zc.get_discount_factor(4))

# Demonstrate interpolation for a non-standard maturity
maturity_lookup = 1.5
print(f"Zero rate for {maturity_lookup} years:", zc.get_zero_rate(maturity_lookup))
print(f"Amount at Maturity for {maturity_lookup} years:", zc.get_AtMat(maturity_lookup))
print(f"Discount factor for {maturity_lookup} years:", zc.get_discount_factor(maturity_lookup))

# Get the complete zero curve data
print("Complete zero curve:", zc.get_zero_curve())

# Create a pandas DataFrame for better data visualization and analysis
zCurve = np.transpose(zc.get_zero_curve())
zc_dataframe = pd.DataFrame(zCurve, columns=['Maturity', 'Discount Factor'])
zc_dataframe.set_index('Maturity', inplace=True)
print("\nZero Curve DataFrame:")
print(zc_dataframe)
zc_dataframe


1-year zero rate: 0.01
1-year discount factor: 0.9900498337491681
2-year discount factor: 0.9607894391523232
3-year discount factor: 0.9139311852712282
4-year discount factor: 0.8521437889662113
Zero rate for 1.5 years: 0.016666666666666535
Amount at Maturity for 1.5 years: 1.0253151205244286
Discount factor for 1.5 years: 0.9753099120283327
Complete zero curve: ([1, 2, 3, 4], [0.9900498337491681, 0.9607894391523232, 0.9139311852712282, 0.8521437889662113])

Zero Curve DataFrame:
          Discount Factor
Maturity                 
1.0              0.990050
2.0              0.960789
3.0              0.913931
4.0              0.852144


,Discount Factor
Maturity,
1.0,0.990050
2.0,0.960789
3.0,0.913931
4.0,0.852144
